In [25]:
# %pip install git+https://github.com/tensorflow/docs

import os.path
from typing import List

import numpy as np
import pandas as pd

from datetime import date
from thefuzz import fuzz

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_colwidth = 160

from utils.configuration import settings
from utils.utils import *
from tft.api import *


# Config

In [26]:
API_KEY: str = settings.api_key
ASSETS_DIR: str = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT: int = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344
RANKED_ID = 1100
PATCH: date = date(2022, 7, 1)

TARGETNAME = 'placement'

In [27]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [28]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,mn3OHPDTDc5CsloPPGlHOQOq0Vqagk66ReC61-8SuXUxqH0,K4FBeziXvtE_px9iFYbp72rXuEhJO_W21lbvRF5VgM4Spw,502mQ2ZxwRwZDIiwzDF_nEvUBsq8qhU9kG_WayPz4qz9DEgFv2cUvXo3b3Yk34UcSAPITT9Zh7zXeA,BC Socks,3552,1657426743000,74,mn3OHPDTDc5CsloPPGlHOQOq0Vqagk66ReC61-8SuXUxqH0,BC Socks,1106,I,239,134,True,False,False,False
1,_U4GJSGv-DEG7MKfJKLmsV-mvefxXMzApJ4tGa_nRQv5sUk,KGjWpE5vcfFpODRSOE4BH7A-3ygXEtFZ6MoXlEKRqXP8_j8,nXZ7c6RVRuEWPvjK_t8dNY5mvebSy0FrzMbupxW_HIbvx8jjl5pK7mSgM60-9I1vaHCrqe1v2AMMuQ,sètsuko,1385,1657395926000,380,_U4GJSGv-DEG7MKfJKLmsV-mvefxXMzApJ4tGa_nRQv5sUk,sètsuko,1599,I,343,160,True,False,False,False
2,oDiXv06lmEaiaWpfRk_9UnckptzvDSU7cm9xezP9OpqmOh4,bJPkEsyiE5upl1f8RxNxWDoh1l62bEQ08YIrK7msKV2kQ9s,OxlKQzWyXu6qQOVkihIKd3BLYh-iAbxMMLUSmNrkqKRrbLPBp08emZS07zn9YxZo97zFRKha30AuxA,captain kookie,4794,1657326115000,362,oDiXv06lmEaiaWpfRk_9UnckptzvDSU7cm9xezP9OpqmOh4,captain kookie,514,I,122,81,False,False,True,False
3,F0FheoSCft_fG9O_CKetV1kygv9j-eujgf8z5Vv2jZsxeH0,2qlMMcJSMHOO6ehsV2_Fk3CqF1Y2FLohbTHIeo16moNbCxM,Kxo_COvGSzgTYYvtuR0nk8gB6NREQ5pJdjSHOa74ZLGgXYTnOAUbgYdRmAriFLJP-yajjPQcbiOPXQ,Clear,3837,1657395926000,147,F0FheoSCft_fG9O_CKetV1kygv9j-eujgf8z5Vv2jZsxeH0,Clear,760,I,144,106,True,False,False,False
4,9BtCw3yqmbey6yGARkIAPCeKeCPF4SqzrwlAtbuFVTpX06s,GxpOZtrCOBmQGwVK-lpxsiE5-YaVPi3Mb2WoaqpiKsiGuA,5EBycu8MnY8t5LmyomNOoVcFt54BEvMeiguhbjrHxFy7_gATw_vph_HkPfrUIPNk9CtRi9z-kiWrvQ,Sphinx,576,1657393039000,72,9BtCw3yqmbey6yGARkIAPCeKeCPF4SqzrwlAtbuFVTpX06s,Sphinx,938,I,225,140,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,bicVkGlwpad_Ahy8HeUW7SCgtpu40D47FJi21MprClE3vm4,521JLtSaP9OJgbT79TDNIlZZycobCEEZNEtWOkzPBxs5Qw,2iaL68PHHTt846wEgYG-Toim_E0Hdw1obob-rUcEtiHCE-v26nt1F4cRCdiRdV3FLXpX9kQOroXlTQ,Ripple Overdrive,1635,1657420466000,64,bicVkGlwpad_Ahy8HeUW7SCgtpu40D47FJi21MprClE3vm4,Ripple Overdrive,753,I,138,103,False,False,False,False
206,UVwLbZlXBLD3fknb2AdgGCbcyH_Zge0cOsNxPE15CTuHfs0,DUPzkyPTTruvYRZxiwTWuJnMKY3pIu56kZA70GPeDnOQoR0,fOv0okB23VegtaIZwEjFFMb_ovNpDQkYFBYK24IBZR_ya0nIOY7d0gb1A0Dsr9SMwW1g7py-9HtRQQ,XII douyu9305388,5410,1657423859068,223,UVwLbZlXBLD3fknb2AdgGCbcyH_Zge0cOsNxPE15CTuHfs0,XII douyu9305388,760,I,264,225,False,False,False,True
207,e34rJaogiO4CE8zLS8ewLMFGPGoNIMIMsB7gDqVH9aPNHIuY,BcLHPFVnuMFT_g6cJdB7LsDocnWKJ3TQyp9mMIY7amMcq-PdOm3O0410,St_wL0UOxf_11zObuZn-7FE-yGRGjfjFaeHlAbCMCESgyqgS636iSJhu5qxxDqBq7XuNlL4I161r8g,Spethom,4904,1657431643000,198,e34rJaogiO4CE8zLS8ewLMFGPGoNIMIMsB7gDqVH9aPNHIuY,Spethom,881,I,152,93,True,False,False,True
208,dOMGV91pp2e7mqHOffMIKZn9lNMpxljwpHtDrHuiKrnF6OYl,ZvgfFlXsNebfxn9dUheDa0H4Dhb-4vVCzSRAkv2NhM28w7VTVrGgyMe8,tiwoB5V6ZKlhgzdc6j2mLIfou3-tf9BopY-5y7acb0ZKdb-GyIWGfhTsEsOJy0W0726IxZMk6YcRfA,WizardTFT,5369,1657433252000,62,dOMGV91pp2e7mqHOffMIKZn9lNMpxljwpHtDrHuiKrnF6OYl,WizardTFT,559,I,167,124,False,False,True,False


# Load unique matches id

In [29]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [30]:
len(uniq_matches_id)

7697

In [31]:
len(matches_asset)

11867

In [32]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [33]:
len(uniq_matches)

7697

## Filter by LATEST_RELEASE version and RANKED_ID game.

In [34]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [35]:
matches_asset[300]['info']['queue_id']

1100

In [36]:
latest_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version']) and (
    RANKED_ID == match['info']['queue_id'])]


In [37]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [38]:
len(latest_matches)

5538

In [39]:
len(latest_patch_matches)

3856

In [40]:
# latest_matches[0]['info']

# Process api details to datasets rows

In [41]:
def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [42]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)

In [43]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4368855050',
 'placement': 4,
 'augment0': 'TFT6_Augment_CyberneticUplink1',
 'augment1': 'TFT7_Augment_PandorasBench',
 'augment2': 'TFT6_Augment_Diversify3',
 'Set7_Astral': 0,
 'Set7_Bruiser': 1,
 'Set7_Dragon': 1,
 'Set7_Evoker': 1,
 'Set7_Guild': 1,
 'Set7_Mage': 2,
 'Set7_Mystic': 1,
 'Set7_Revel': 0,
 'Set7_Trainer': 1,
 'Set7_Whispers': 2,
 'TFT7_Heimerdinger': 2,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Heimerdinger_item0': 'NeedlesslyLargeRod',
 'TFT7_Nami': 2,
 'TFT7_Nami_item0': 'Shroud',
 'TFT7_Lulu': 2,
 'TFT7_Lulu_item0': 'StatikkShiv',
 'TFT7_Ryze': 2,
 'TFT7_Ryze_item0': 'SpearOfShojin',
 'TFT7_Sylas': 2,
 'TFT7_Sylas_item0': 'WarmogsArmor',
 'TFT7_DragonPurple': 2,
 'TFT7_DragonPurple_item0': 'HextechGunblade',
 'TFT7_DragonPurple_item1': 'WarmogsArmor',
 'TFT7_Sona': 2,
 'TFT7_Sona_item0': 'MageEmblemItem',
 'TFT7_Sona_item1': 'JeweledGauntlet',
 'TFT7_TrainerDragon': 3,
 'TFT7_TrainerDragon_item0': 'TrainerSn

In [44]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)

In [45]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Astral,Set7_Bruiser,Set7_Dragon,Set7_Evoker,Set7_Guild,...,TFT7_Soraka_item2,TFT7_AurelionSol_item2,TFT7_Braum_item2,TFT7_Karma_item1,TFT7_Thresh_item1,TFT7_Lillia_item2,TFT7_Karma_item2,TFT7_Senna_item2,TFT7_Thresh_item2,TFT7_Sett_item2
0,NA1_4368855050,4,TFT6_Augment_CyberneticUplink1,TFT7_Augment_PandorasBench,TFT6_Augment_Diversify3,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4368855050,1,TFT6_Augment_Distancing,TFT6_Augment_SecondWind2,TFT7_Augment_GuildEmblem2,NaN,1.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4368855050,5,TFT6_Augment_ItemGrabBag1,TFT6_Augment_TargetDummies,TFT6_Augment_CyberneticImplants3,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4368855050,7,TFT7_Augment_ShimmerscaleEmblem,TFT6_Augment_Diversify2,TFT6_Augment_MakeshiftArmor3,NaN,1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4368855050,6,TFT6_Augment_Weakspot,TFT6_Augment_ComponentGrabBag,TFT6_Augment_CyberneticUplink3,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,NA1_4355962932,2,TFT6_Augment_PandorasItems,TFT6_Augment_CyberneticImplants2,TFT7_Augment_FirstAidKit2,NaN,1.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44300,NA1_4355962932,8,TFT6_Augment_ThrillOfTheHunt1,TFT7_Augment_FirstAidKit2,TFT7_Augment_WarriorEmblem,NaN,1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44301,NA1_4355962932,6,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_TradeSectorPlus,TFT6_Augment_SecondWind2,NaN,0.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44302,NA1_4355962932,5,TFT6_Augment_SecondWind1,TFT7_Augment_FirstAidKit2,TFT6_Augment_SecondWind2,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Astral,Set7_Bruiser,Set7_Dragon,Set7_Evoker,Set7_Guild,...,TFT7_Soraka_item2,TFT7_AurelionSol_item2,TFT7_Braum_item2,TFT7_Karma_item1,TFT7_Thresh_item1,TFT7_Lillia_item2,TFT7_Karma_item2,TFT7_Senna_item2,TFT7_Thresh_item2,TFT7_Sett_item2
0,NA1_4368855050,4,TFT6_Augment_CyberneticUplink1,TFT7_Augment_PandorasBench,TFT6_Augment_Diversify3,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4368855050,1,TFT6_Augment_Distancing,TFT6_Augment_SecondWind2,TFT7_Augment_GuildEmblem2,NaN,1.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4368855050,5,TFT6_Augment_ItemGrabBag1,TFT6_Augment_TargetDummies,TFT6_Augment_CyberneticImplants3,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4368855050,7,TFT7_Augment_ShimmerscaleEmblem,TFT6_Augment_Diversify2,TFT6_Augment_MakeshiftArmor3,NaN,1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4368855050,6,TFT6_Augment_Weakspot,TFT6_Augment_ComponentGrabBag,TFT6_Augment_CyberneticUplink3,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,NA1_4355962932,2,TFT6_Augment_PandorasItems,TFT6_Augment_CyberneticImplants2,TFT7_Augment_FirstAidKit2,NaN,1.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44300,NA1_4355962932,8,TFT6_Augment_ThrillOfTheHunt1,TFT7_Augment_FirstAidKit2,TFT7_Augment_WarriorEmblem,NaN,1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44301,NA1_4355962932,6,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_TradeSectorPlus,TFT6_Augment_SecondWind2,NaN,0.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44302,NA1_4355962932,5,TFT6_Augment_SecondWind1,TFT7_Augment_FirstAidKit2,TFT6_Augment_SecondWind2,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [47]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [48]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)

# Output dataframes

In [49]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)

# End